In [5]:
import pandas as pd

from courses.models import Course, Registration
from users.models import User

df = pd.read_excel("course info.xlsx", sheet_name="تجربه‌گرها")

# leadership = CourseType.objects.get_or_create(
#     name="leadership",
#     name_fa="پیش‌وایی",
#     category=1
# )[0]
# relationship = CourseType.objects.get_or_create(
#     name="relationship",
#     name_fa="رابطه",
#     category=1
# )[0]
# pileh = CourseType.objects.get_or_create(
#     name="pileh",
#     name_fa="پیل",
#     category=1
# )[0]
# heart_of_matter = CourseType.objects.get_or_create(
#     name="heart of matter",
#     name_fa="جان کلام",
#     category=2
# )[0]
# accomplishment = CourseType.objects.get_or_create(
#     name="accomplishment",
#     name_fa="دستاورد",
#     category=2
# )[0]
# confront = CourseType.objects.get_or_create(
#     name="confront",
#     name_fa="مواجهه",
#     category=2
# )[0]
# transforming_year = CourseType.objects.get_or_create(
#     name="transforming year",
#     name_fa="تحویل سال",
#     category=2
# )[0]

# course_to_create = []
# course_to_create = course_to_create + [(leadership, i+1) for i in range(20)]
# course_to_create = course_to_create + [(relationship, i+1) for i in range(5)]
# course_to_create = course_to_create + [(pileh, i+1) for i in range(6)]
# course_to_create = course_to_create + [(heart_of_matter, i+1) for i in range(3)]
# course_to_create = course_to_create + [(accomplishment, i+1) for i in range(1)]
# course_to_create = course_to_create + [(confront, i+1) for i in range(3)]
# course_to_create = course_to_create + [(transforming_year, i+1) for i in range(3)]
# done_courses = Course.objects.bulk_create(
#     Course(
#         course_type=course_type,
#         number = i,
#         course_name=f"{course_type} {i + 1}"
#     ) for course_type, i in course_to_create
# )

In [ ]:
from tqdm import tqdm


def normalize_phone(phone):
    phone = str(phone).strip()
    if len(phone) == 11 and phone.startswith("09"):
        return "+98" + phone[1:]
    return None


# df['normalized_phone'] = df['fixed phone'].apply(normalize_phone)
# good_phones = df['normalized_phone'].dropna().unique()

good_phone = []
bad_phone = []
for phone in df["fixed phone"]:
    if len(str(phone).strip()) == 11 and str(phone).startswith("09"):
        good_phone.append("+989" + phone[2:])
    else:
        bad_phone.append(phone)
good_phone = list(set(good_phone))
bad_phone = list(set(bad_phone))

users_dic = {user.phone_number: user for user in User.objects.filter(phone_number__in=good_phone)}


def get_gender(gender):
    if gender in ["M", "m", "مرد"]:
        return 1
    elif gender in ["F", "f", "زن"]:
        return 2
    return None


def convert_to_english(text):
    to_english = {
        "۰": "0",
        "۱": "1",
        "۲": "2",
        "۳": "3",
        "۴": "4",
        "۵": "5",
        "۶": "6",
        "۷": "7",
        "۸": "8",
        "۹": "9",
        "٠": "0",
        "١": "1",
        "٢": "2",
        "٣": "3",
        "٤": "4",
        "٥": "5",
        "٦": "6",
        "٧": "7",
        "٨": "8",
        "٩": "9",
    }
    for digit, english_digit in to_english.items():
        text = text.replace(digit, english_digit)
    return text


def get_national_id(national_id):
    if not national_id:
        return None
    if len(str(national_id).strip()) == 8:
        return "00" + convert_to_english(str(national_id)).strip()
    return convert_to_english(str(national_id)).strip()


def get_status(status):
    if status in ["انتقال به دوره های بعد", "انصراف قبل از دوره", "غایب", "انصراف پیش از دوره"]:
        return 2, 3
    elif status in ["انصراف روز اول", "انتخاب نماندن"]:
        return 3, 3
    elif status in ["انصراف روز اول - غایب", "ترک دوره"]:
        return 4, 3
    elif status in ["تیم اجرایی"]:
        return 7, 3
    elif status in ["مهمان"]:
        return 8, 3
    elif status in ["شرکت کننده", "حاضر در دوره"]:
        return 9, 3
    elif status in ["ثبت‌نام", "در انتظار پرداخت"]:
        return 1, 3
    elif status in ["حاضر در دوره - انتظار پرداخت"]:
        return 9, 2
    elif status in ["حاضر در دوره - عدم سررسید"]:
        return 9, 1
    elif status in ["حاضر در دوره - تسویه"]:
        return 9, 3
    return 9, 3


done_registrations = Registration.objects.filter(user__in=users_dic.values())
registration_dic = {f"{reg.user.id}-{reg.course.id}": reg for reg in done_registrations}
course_dic = {
    f"{course.course_type.name_fa}-{course.number}": course
    for course in Course.objects.select_related("course_type").all()
}

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    phone = normalize_phone(row["fixed phone"])
    first_name = str(row.get("نام", "")).strip()
    last_name = str(row.get("نام‌خانوادگی", "")).strip()
    if not phone:
        continue
    if phone in users_dic:
        user = users_dic[phone]
        if user.first_name != first_name or user.last_name != last_name:
            print("inconsistent user data", index, phone, user.full_name, first_name, last_name)
            continue
    else:
        user = User.objects.create(
            username=phone,
            phone_number=phone,
            first_name=first_name,
            last_name=last_name,
            telegram_id=row.get("تلگرام", ""),
            email=row.get("ایمیل", ""),
            profession=row.get("حرفه", ""),
            age=row.get("سن", None),
            gender=get_gender(row.get("جنسیت", None)),
            national_id=get_national_id(row.get("کد ملی", "")),
            english_first_name=row.get("Name", ""),
            english_last_name=row.get("Surname", ""),
            referer_name=row.get("معرف", None),
        )
        users_dic[user.phone_number] = user
    course_name = row["نام دوره"]
    course_number = int(row["شماره دوره"])
    course = course_dic.get(f"{course_name}-{course_number}", None)
    if not course:
        print(f"Course not found: {course_name} - {course_number}", index, phone)
        continue
    status, payment_status = get_status(row.get("وضعیت", None))
    tuition = row.get("مبلغ دوره", 0)
    if registration_dic.get(f"{user.id}-{course.id}", None) is not None:
        continue
    try:
        Registration.objects.create(
            user=user,
            course=course,
            status=status,
            payment_status=payment_status,
            tuition=tuition,
        )
        s
    except:
        print("Error", phone, user.full_name, row.get("نام"))

  6%|███▊                                                        | 130/2048 [00:00<00:03, 637.96it/s]

inconsistent user data 14 +989192386071 آزاده اعتصامی صدری آزاده اعتصامی
inconsistent user data 69 +989122822695 علی اکبر سلطانلو علی اکبر سلطان لو
inconsistent user data 87 +989224830197 سایه زارعی تودشکی سایه زارعی
inconsistent user data 93 +989124493681 مسعود ملک محمدی بیدهندی مسعود ملک محمدی
inconsistent user data 143 +989126798943 امین عارف پوراکبری امین عارف


 13%|███████▌                                                    | 258/2048 [00:00<00:03, 548.45it/s]

inconsistent user data 170 +989394310605 مارال احمدی آرزو نصیری
inconsistent user data 183 +989122822695 علی اکبر سلطانلو علی اکبر سلطان لو
inconsistent user data 184 +989126798943 امین عارف پوراکبری امین عارف
inconsistent user data 190 +989192386071 آزاده اعتصامی صدری آزاده اعتصامی
inconsistent user data 200 +989122091903 محمد علی صابری محمدعلی صابری
inconsistent user data 212 +989124493681 مسعود ملک محمدی بیدهندی مسعود ملک محمدی
inconsistent user data 214 +989203109686 محمدعلى طائبى محمدعلی طائبی
inconsistent user data 239 +989123441962 امیرعلی حسینخان امیرعلی حسین‌خان
inconsistent user data 244 +989390372267 آرش آل بیگی غلامی آرش آل‌بیگی
inconsistent user data 250 +989366205771 سعید عالم ورزنه اصفهانی سعید عالم


 19%|███████████▏                                                | 380/2048 [00:00<00:03, 550.90it/s]

...

inconsistent user data 1182 +989143159445 آرش سروی آرمین سروی
inconsistent user data 1214 +989210829141 اشکان فراهانی فائزه دادخواهی اصل
inconsistent user data 1249 +989127224421 مجید کرمی مجید کریمی
inconsistent user data 1271 +989130256846 محمد یاسین پارسی محمدیاسین پارسی
inconsistent user data 1309 +989122822695 علی اکبر سلطانلو علی اکبر سلطان لو
inconsistent user data 1322 +989123370623 سمیه مرادلو آرش شیخی
inconsistent user data 1329 +989224830197 سایه زارعی تودشکی سایه زارعی


 82%|███████████████████████████████████████████████▌          | 1679/2048 [00:02<00:00, 1099.42it/s]

inconsistent user data 1465 +989121386329 سیدمحمدرضا نجفی سید محمدرضا نجفی
inconsistent user data 1480 +989121784378 مهدیار کوکب زاده مهدی یار کوکب زاده
inconsistent user data 1489 +989010131603 سپیده زیادی پناه سپیده پناه
inconsistent user data 1500 +989196208122 آرش کاظم تاش همسر آرش کاظم تاش
inconsistent user data 1502 +989127192230 آرش خسرویان آرش خسرویان عرب
inconsistent user data 1520 +989123798829 امیر عباس فتوت پریسا باقریان


100%|███████████████████████████████████████████████████████████| 2048/2048 [00:03<00:00, 656.27it/s]

inconsistent user data 1728 +989121070729 امین سمیع‌زاده امین سمیع زاده
inconsistent user data 1733 +989157571200 امیرحسین غفوریان حسین غفوریان
inconsistent user data 1741 +989125984866 مهدی طالبی شاهرخ طالبی
inconsistent user data 1744 +989027223818 حسن کرمی علی ملک نسب
inconsistent user data 1764 +989124922268 میلاد میرزازاده میلاد میرزا زاده


In [4]:
User.objects.get(username="usertest1")

DoesNotExist: User matching query does not exist.